# Big Code Bench Mistral

In [8]:
from human_eval.data import write_jsonl, read_problems, BIGCODE_TEST
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import time, datetime
from mistralai import Mistral
import json
from typing import Iterable, Dict
import gzip

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def get_import_statements(code_str: str) -> str:
    """
    Keeps lines that begin with 'import' or 'from' from a multiline string.
    """
    lines = code_str.splitlines()
    filtered_lines = []
    for line in lines:
        stripped_line = line.strip()
        if stripped_line.startswith('import ') or stripped_line.startswith('from '):
            filtered_lines.append(line)
    res = '\n'.join(filtered_lines) + '\n'
    s = 'from typing import List'
    if s not in res:
        res = s + '\n' + res
    return res

# Example usage
code_with_imports = '''import collections
import random
import string

def task_func(length=100):
    return 1
'''
print( get_import_statements(code_with_imports) )

from typing import List
import collections
import random
import string



In [3]:
def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))


def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


def read_problems(evalset_file: str) -> Dict[str, Dict]:
    return {task["task_id"]: task for task in stream_jsonl(evalset_file)}

In [4]:
def is_full_function(input_str: str) -> str:
    '''
    Determine if input string is a Python function
    using simple heuristics (all that's needed)
    '''
    lines = input_str.splitlines()
    for line in lines:
        if line.startswith('def '):
            return True
    return False


def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if 'def ' in text and s not in text:
        text = s + '\n\n' + text
    return text

string = 'def my_func(input):\n    return output'
print(add_import(string))

from typing import List

def my_func(input):
    return output


In [18]:
def get_response(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model = model,
                messages = [{
                        "role": "user",
                        "content": prompt, 
                    }, ]
                )
    return chat_response.choices[0].message.content

In [6]:
def read_jsonl(file_path):
    '''
    Read a JSONL file.
    Parameters:
        file_path (str): The path to the JSONL file.
    Returns:
        List[dict]: A list of dictionaries representing the JSON objects.
    '''
    data = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line:
                data.append(json.loads(line))
    return data

In [9]:
tasks = read_problems(BIGCODE_TEST)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 10:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: BigCodeBench/0 <class 'str'>

task_id:
BigCodeBench/0

test:


import unittest
from unittest.mock import patch
from random import seed, shuffle
import itertools
class TestCases(unittest.TestCase):
    def test_default_numbers(self):
        # Test with default number range (1 to 10) to check that the result is a positive float.
        result = task_func()
        self.assertIsInstance(result, float)
        self.assertGreater(result, 0)
    def test_custom_list(self):
        # Test with a custom list of small positive integers to ensure proper handling and positive result.
        result = task_func([1, 2, 3])
        self.assertIsInstance(result, float)
        self.assertGreater(result, 0)
    def test_negative_numbers(self):
        # Test with negative numbers to verify the function handles and returns a positive result.
        result = task_func([-3, -2, -1])
        self.assertIsInstance(result, float)
        self.assertGreater(result, 

In [10]:
task = tasks['BigCodeBench/15']

# smaller models ouput additional trash with this propmt (before and after the code)
completion_prompt = 'Complete the following Python code:\n'

completion_prompt = '''1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:\n\n'''
print(completion_prompt + task['prompt'])

1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import subprocess
import csv
import os

def task_func(commands_file_path, output_dir_path):
    """
    Execute a list of shell commands read from a CSV file and save the outputs in separate files.
    Each command's output is written to a unique file in the specified output directory.
    If a command fails, the error message along with the exit code is appended to the respective output file.

In [11]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [37]:
mistral_7b = 'open-mistral-7b'
mixtral = 'open-mixtral-8x7b'
mistral_3b = 'ministral-3b-latest'
mistral_8B = 'ministral-8b-latest'
codestral_mamba = 'open-codestral-mamba'
cnemo = 'open-mistral-nemo'
codestral = 'codestral-latest'
mistral_small = 'mistral-small-latest'

model = codestral
print(get_response(model, completion_prompt.format(task['prompt']) + task['test']))

def task_func(commands_path, output_dir):
    """
    Executes commands from a CSV file and writes the output to files in the output directory.

    Args:
        commands_path (str): Path to the CSV file containing commands.
        output_dir (str): Path to the directory where output files will be saved.

    Returns:
        list: List of output file names.
    """
    output_files = []
    try:
        with open(commands_path, newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                command = row[0]
                output_file = os.path.join(output_dir, f"{len(output_files)}.txt")
                output_files.append(output_file)
                try:
                    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    with open(output_file, 'w') as f:
                        f.write(result.stdout.decode())
                except subprocess.CalledProcessE

In [38]:
model

'codestral-latest'

In [39]:
model_nickname = model
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'logs/{model_nickname}_samples_{time_stamp}.jsonl'
file_name

'logs/codestral-latest_samples_20250124_110832_6712.jsonl'

In [40]:
num_samples_per_task = 1
completions = []
is_full_func = []

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == 'BigCodeBench/7':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = completion_prompt + task_body['prompt']
        try:
            completion  = get_response( model, full_prompt )
        except Exception as e:
            completion = f"Error generating a completion:\n{e}"

        completion = remove_triple_backticks_at_edges(completion.strip())
        is_full = is_full_function(completion)
        is_full_func.append(is_full)
        if is_full:
            completion = get_import_statements(task_body['prompt']) + '\n' + completion
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Prompt:\n', full_prompt, '\n', '-'*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*75, '\n', sep='')

BigCodeBench/0
Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import itertools
from random import shuffle

def task_func(numbers=list(range(1, 3))):
    """
    Calculates the average of the sums of absolute differences between each pair of consecutive numbers 
    for all permutations of a given list. Each permutation is shuffled before calculating the differences.

    Args:
    - numbers (list): A list of numbers. Default is num

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import subprocess
import csv
import os

def task_func(commands_file_path, output_dir_path):
    """
    Execute a list of shell commands read from a CSV file and save the outputs in separate files.
    Each command's output is written to a unique file in the specified output directory.
    If a command fails, the error message along with the exit code is appended to the respective outp

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import numpy as np
from itertools import zip_longest

def task_func(l1, l2,THRESHOLD = 0.5):
    """
    Alternates elements from two numeric lists, calculates the absolute difference of each 
    element from a predefined threshold, and returns the element closest to this threshold.
    
    Parameters:
    l1 (list): The first input list containing numeric values.
    l2 (list): The 

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import numpy as np
from functools import reduce

def task_func(list_of_pairs):
    """ 
    Calculate the product of the second values in each tuple in a list of tuples and return the product as a single-element numeric array.
    
    Parameters:
    list_of_pairs (list): A list of tuples, where the first element is the category 
                          and the second element is the

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import numpy as np
import seaborn as sns

def task_func(df):
    """
    Describe a dataframe and draw a distribution chart for each numeric column after replacing the NaN values with the average of the column.

    Parameters:
    df (DataFrame): The pandas DataFrame.

    Returns:
    tuple: A tuple containing:
        - DataFrame: A pandas DataFrame with statistics. This includes co

...

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import ast
import os
import glob

# Constants
DIRECTORY = 'data'

def task_func(directory):
    """
    Convert all Unicode string representations of dictionaries in all text files 
    in the specified directory to Python dictionaries.

    Parameters:
    directory (str): The path to the directory containing the text files.

    Returns:
    list: A list of dictionaries extracted fro

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

from nltk.tokenize import RegexpTokenizer
from string import punctuation
import csv
import os

PUNCTUATION = set(punctuation)

def task_func(text, filename):
    """
    Save all words in a text beginning with the "$" character in a CSV file, excluding any words that are solely composed of punctuation characters.

    Parameters:
    text (str): The input text.
    filename (str): The 

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import subprocess
import os
import threading

def task_func(script_path: str, timeout: int = 60) -> str:
    """
    Execute a specified python code with a given timeout. If the script execution exceeds the timeout, it is terminated.

    Parameters:
    - script_path (str): The path to the Python code to be executed.
    - timeout (int): The maximum allowed time (in seconds) for the s

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import re
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests


def task_func(myString):
    """
    Extracts a URL from a given string and retrieves the title of the web page from that URL. If no valid URL is found,
    or the URL does not result in a successful web page fetch, returns an appropriate error message.

    Parameters:
    myString (str): The s

Prompt:
1. Complete the following starter code consisting of import statements, function signature and docstring.
2. Your output must include only the function body with proper Python code and indentation.
3. Do not include phrases like “Completion:” or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, no code fences (```), and no other content before or after the function body.
5. Stop generating immediately after the return statement or final line of the function body.

Starter Code:

import os
import hashlib
import base64

def task_func(password, PREFIX="ME", SALT_LENGTH=16):
    """
    Generates a hashed password by concatenating a given password with a prefix and a generated salt,
    and then hashing the combined string using SHA256. The hashed result is then encoded in base64.

    Parameters:
    - password (str): The password string to hash.
    - PREFIX (st

In [41]:
# if the above cell has not been run
#file_name = 'logs2/solar_10B_samples_20241127_232205_5243.jsonl'

In [42]:
# count how many full funcs were generated vs. only func bodies without func signatures
from collections import Counter
Counter(is_full_func)

Counter({False: 470, True: 30})

In [43]:
evaluate_functional_correctness(file_name, k=[1], mode='big_code')

Reading samples...


500it [00:00, 9304.75it/s]


Running test suites...


100%|██████████████████████████████████████████████████████████████████| 500/500 [00:33<00:00, 15.11it/s]


Writing results to logs/codestral-latest_samples_20250124_110832_6712.jsonl_results.jsonl...


100%|███████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 89671.70it/s]


{'pass@1': 0.128}

## Getting Model Temperature and Top_p
According to [Mistral docs](https://docs.mistral.ai/api/#tag/chat/operation/chat_completion_v1_chat_completions_post):
* Default top_p is 1
* To find out about the default temperature, one neexs to call the `/models` endpoint to retrieve the appropriate value.

In [70]:
# calling the /models endpoint
import requests

api_key = os.environ["MISTRAL_API_KEY"]

# Correct API endpoint
url = "https://api.mistral.ai/v1/models"

# Headers with the correct authorization format
headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/json"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an error for HTTP 4xx/5xx codes
    print("Response:")
    print(response.json())
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    print(f"Response: {response.text}")
except Exception as e:
    print(f"An error occurred: {e}")


Response:
{'object': 'list', 'data': [{'id': 'ministral-3b-2410', 'object': 'model', 'created': 1736121321, 'owned_by': 'mistralai', 'capabilities': {'completion_chat': True, 'completion_fim': False, 'function_calling': True, 'fine_tuning': True, 'vision': False}, 'name': 'ministral-3b-2410', 'description': 'Official ministral-3b-2410 Mistral AI model', 'max_context_length': 131072, 'aliases': ['ministral-3b-latest'], 'deprecation': None, 'default_model_temperature': 0.3, 'type': 'base'}, {'id': 'ministral-3b-latest', 'object': 'model', 'created': 1736121321, 'owned_by': 'mistralai', 'capabilities': {'completion_chat': True, 'completion_fim': False, 'function_calling': True, 'fine_tuning': True, 'vision': False}, 'name': 'ministral-3b-2410', 'description': 'Official ministral-3b-2410 Mistral AI model', 'max_context_length': 131072, 'aliases': ['ministral-3b-2410'], 'deprecation': None, 'default_model_temperature': 0.3, 'type': 'base'}, {'id': 'ministral-8b-2410', 'object': 'model', 'cr

In [72]:
response.json()

{'object': 'list',
 'data': [{'id': 'ministral-3b-2410',
   'object': 'model',
   'created': 1736121321,
   'owned_by': 'mistralai',
   'capabilities': {'completion_chat': True,
    'completion_fim': False,
    'function_calling': True,
    'fine_tuning': True,
    'vision': False},
   'name': 'ministral-3b-2410',
   'description': 'Official ministral-3b-2410 Mistral AI model',
   'max_context_length': 131072,
   'aliases': ['ministral-3b-latest'],
   'deprecation': None,
   'default_model_temperature': 0.3,
   'type': 'base'},
  {'id': 'ministral-3b-latest',
   'object': 'model',
   'created': 1736121321,
   'owned_by': 'mistralai',
   'capabilities': {'completion_chat': True,
    'completion_fim': False,
    'function_calling': True,
    'fine_tuning': True,
    'vision': False},
   'name': 'ministral-3b-2410',
   'description': 'Official ministral-3b-2410 Mistral AI model',
   'max_context_length': 131072,
   'aliases': ['ministral-3b-2410'],
   'deprecation': None,
   'default_mode

In [79]:
for i in response.json()['data']:
    print(f"Model: {i['id']}. Temperature: {i['default_model_temperature']}")

Model: ministral-3b-2410. Temperature: 0.3
Model: ministral-3b-latest. Temperature: 0.3
Model: ministral-8b-2410. Temperature: 0.3
Model: ministral-8b-latest. Temperature: 0.3
Model: open-mistral-7b. Temperature: 0.7
Model: mistral-tiny. Temperature: 0.7
Model: mistral-tiny-2312. Temperature: 0.7
Model: open-mistral-nemo. Temperature: 0.3
Model: open-mistral-nemo-2407. Temperature: 0.3
Model: mistral-tiny-2407. Temperature: 0.3
Model: mistral-tiny-latest. Temperature: 0.3
Model: open-mixtral-8x7b. Temperature: 0.7
Model: mistral-small. Temperature: 0.7
Model: mistral-small-2312. Temperature: 0.7
Model: open-mixtral-8x22b. Temperature: 0.7
Model: open-mixtral-8x22b-2404. Temperature: 0.7
Model: mistral-small-2402. Temperature: 0.7
Model: mistral-small-2409. Temperature: 0.7
Model: mistral-small-latest. Temperature: 0.7
Model: mistral-medium-2312. Temperature: 0.7
Model: mistral-medium. Temperature: 0.7
Model: mistral-medium-latest. Temperature: 0.7
Model: mistral-large-2402. Temperature